# Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()


In [ ]:
PROJECT_ID = '<GCP-PROJECT-ID>'
REGION = '<GCP-REGION>'
BUCKET = '<GCP-BUCKET>'
FILENAME_TXT = '<FILENAME-TXT>'
FILENAME_PDF = '<FILENAME-PDF>'


# Install Dependencies

In [ ]:
!pip install --upgrade shapely
!pip install --upgrade google-cloud-aiplatform # Vertex AI module
!pip install --upgrade google-cloud-storage # Cloud Storage module
!pip install --upgrade fpdf # Module to convert .txt to .pdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.3/321.3 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 101.2 MB/s eta 0:00:00
  Attempting uninstall: shapely
    Found existing installation: shapely 2.0.1
    Uninstalling shapely-2.0.1:
      Successfully uninstalled shapely-2.0.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 2.8.0
    Uninstalling google-cloud-storage-2.8.0:
      Successfully uninstalled google-cloud-storage-2.8.0


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=fcafe2fb1bd688859cc52443a354367f6ba1c106ffe5bbbc18f949e1a6b73f87
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


# Libraries

In [ ]:
import vertexai
import random
from vertexai.preview.language_models import ChatModel, InputOutputTextPair
from vertexai.language_models import TextGenerationModel

# Generate Customer Scenario

In [ ]:
personas = ['impatient','dissatisfied','angry','frugal','clueless','defective object', 'feature request']
areas = ["Compute", "Storage", "Database", "Networking", "Big Data", "Machine Learning", "Internet of Things", "Serverless", "Security", "Management"]

selected_persona = random.choice(personas)
selected_area = random.choice(areas)

issue = f"You are a {selected_persona} customer and you are experiencing a problem related to {selected_area} on Google Cloud. \nWhen prompted, you will provide a detailed description of the problem you are facing, including any relevant context or issues you have encountered. \nPlease note that you should keep the dialogue brief but conversational and ask questions to solve your issue."

print(issue)

You are a feature request customer and you are experiencing a problem related to Compute on Google Cloud. 
When prompted, you will provide a detailed description of the problem you are facing, including any relevant context or issues you have encountered. 
Please note that you should keep the dialogue brief but conversational and ask questions to solve your issue.


# Customer Interaction

With the above context, kindly interact with the customer. Type exit or EXIT to stop interaction

In [ ]:
# Initiate Vertex AI
vertexai.init(project=PROJECT_ID, location=REGION)
chat_model = ChatModel.from_pretrained("chat-bison@001")
parameters = {
    "temperature": 0.5,
    "max_output_tokens": 256,
    "top_p": 0.8,
    "top_k": 40
}

# Start Chat Model
chat = chat_model.start_chat(context = issue)



# Track converation
conversation = ""
while True:
    user_input = input("Engineer: ")
    if user_input.lower() == 'exit':
        break

    response = chat.send_message(user_input)
    conversation += "Engineer: " + user_input + "\n"
    conversation += "Customer: " + str(response) + "\n"
    print("Customer: " + str(response))

print('##########')
print("Conversation End")



Engineer: my network is down. help
Customer: I'm sorry to hear that your network is down. I can help you troubleshoot the issue. Can you tell me what kind of network you have?
Engineer: I am unable to create a docketr build for my vertex ai workbench pipeline
Customer: I'm sorry to hear that you're having trouble creating a Docker build for your Vertex AI Workbench pipeline. I can help you troubleshoot the issue. Can you tell me what steps you've taken so far?
Engineer: exit
##########
Conversation End


# Feedback

In [ ]:
model = TextGenerationModel.from_pretrained("text-bison@001")
response = model.predict(
    f"""Provide constructive feedback to the Customer Engineer based on : {conversation}. Make a bulleted list of items for a. What went well ?, b. What could have been improved in this interaction? and c. Any additional tips
""",
    **parameters
)
feedback = response.text
print(f"Feedback from model: {feedback}")

# Work on customer feedback


Feedback from model: **What went well:**
* The customer engineer was polite and helpful.
* The customer engineer was able to identify the issue and provide a solution.
* The customer engineer was able to follow up with the engineer to ensure that the issue was resolved.

**What could have been improved:**
* The customer engineer could have been more proactive in asking questions to understand the engineer's issue.
* The customer engineer could have provided more detailed instructions on how to troubleshoot the issue.
* The customer engineer could have offered to help the engineer create a Docker build for their Vertex AI Workbench pipeline.

**Additional tips:**
* When troubleshooting a technical issue, it is important to be clear and concise in your communication.
* It is also important to be patient and understanding.
* If you are not sure how to help someone, it is always better to ask for help than to provide incorrect information.


## Save Interaction as .pdf or .txt

Files will be saved locally

### Save as TXT

In [ ]:
# Create a new text file using input from a "conversation" variable
content = " "
content += "########## Issue ############\n" + issue + "\n"
content += "\n ########## Conversation ##########\n" + conversation + "\n"
content += "\n ############ Feedback ############\n" + feedback + "\n"

with open(FILENAME_TXT, "w") as f:
    f.write(content)

print("TXT Saved")

TXT Saved


### Save as PDF

In [ ]:
from fpdf import FPDF

# Create a new PDF document
pdf = FPDF()

# Open the text file in read mode
with open(FILENAME_TXT, "r") as f:
    # Read the content from the file
    content = f.read()

# Add a page and set the font
pdf.add_page()
pdf.set_font('Arial', size=12)

# Set the text color and insert the content into the PDF
pdf.set_text_color(0, 0, 0)  # Black text color
pdf.multi_cell(0, 10, txt=content)

# Save the PDF file
pdf.output(FILENAME_PDF)

print("PDF Saved")

PDF Saved


## Write to Cloud Bucket

Kindly view `gencapcss` bucket to see files

### Save PDF

In [ ]:
import os
from google.cloud import storage

# Get the bucket name
bucket_name = BUCKET

# Create a storage client
client = storage.Client()

# Create a blob object
blob = client.bucket(bucket_name).blob(FILENAME_PDF)

# Upload the file
blob.upload_from_filename(FILENAME_PDF)
print("PDF Uploaded to Bucket")


PDF Uploaded to Bucket


### Save TXT

In [ ]:
# Create a blob object
blob = client.bucket(bucket_name).blob(FILENAME_TXT)

# Upload the file
blob.upload_from_filename(FILENAME_TXT)
print("TXT Uploaded to Bucket")

TXT Uploaded to Bucket
